In [ ]:
import os
import requests
import pandas as pd
from urllib.parse import urlparse
from dotenv import load_dotenv
from datetime import datetime

In [ ]:
load_dotenv()
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
HEADERS = {"Authorization": f"token {GITHUB_TOKEN}"}

In [ ]:
REQUIRED_PATHS = ["retos_sesion_01/ejercicio_01.txt", "retos_sesion_01/ejercicio_02.txt"]

In [ ]:
repo_url = "https://github.com/taicoding/psg-example"

In [ ]:
def extract_owner_repo(repo_url):
    parts = urlparse(repo_url).path.strip("/").split("/")
    return parts[0], parts[1]

In [ ]:
max_date = datetime.fromisoformat("2025-04-26")

In [ ]:
def check_file_exists(owner, repo, path):
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{path}"
    response = requests.get(url, headers=HEADERS)
    return response.status_code == 200

In [ ]:
def check_commit_dates(owner, repo, path, max_date):
    url = f"https://api.github.com/repos/{owner}/{repo}/commits"
    params = {"path": path, "per_page": 100}
    response = requests.get(url, headers=HEADERS, params=params)
    if response.status_code != 200:
        print(f"Error: {response.status_code} - {response.text}")
        return None

    commits = response.json()

    if not commits:
        return 0
    last_commit = commits[-1]
    file_date = datetime.fromisoformat(last_commit["commit"]["committer"]["date"])
    
    if file_date.date() <= max_date.date():
        return 1
    return -1

In [ ]:
def load_template(path="templates/retos_sesion_01.md"):
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

In [ ]:
def fill_template(template, data):
    return template.format(**data)

In [ ]:
def summary_preview(missing_files, expected_files, file_dates):
    template_data={}
    missing_count = len(missing_files)
    template_data['missing_files'] = missing_count
    files_in_format = expected_files - missing_count
    template_data['files_in_format'] = files_in_format
    folder_format = "✅ Correcta"
    delivery_status = "✅"
    if files_in_format == 0:
        folder_format = "❌ No se encontró los retos"
    if files_in_format > 0 and files_in_format < expected_files:
        folder_format = "⚠️ No se encontró algunos retos"
    template_data['folder_format'] = folder_format
    check_delivery_status = sum(file_dates)
    if check_delivery_status == 0:
        delivery_status = "❌ Incumplido"
    delivery_status = "⚠️ Observado por archivos fuera de fecha."
    template_data["delivery_status"] = delivery_status
    status = "Revision pendiente"
    template_data["status"] = status
    comment = ""
    template_data["comment"] = comment
    
    final_message = "Aún no concluimos la revisión 😅 Gracias por tu paciencia."
    template_data["final_message"] = final_message
    template = load_template()
    issue_content = fill_template(template, template_data)
    
    return issue_content 
    

In [ ]:
def create_issue(owner, repo, body, title):
    url = f"https://api.github.com/repos/{owner}/{repo}/issues"
    payload = {"title": title, "body": body}
    response = requests.post(url, headers=HEADERS, json=payload)
    if response.status_code == 201:
        print(f"Issue creado en {owner}/{repo}")
    else:
        print(f"Error al crear issue en {owner}/{repo}: {response.status_code} - {response.text}")


In [ ]:
owner, repo = extract_owner_repo(repo_url)
print(f"Revisando: {owner}/{repo}")
missing_files = []
dates_review = []
expected_files = 2

for path in REQUIRED_PATHS:
    if not check_file_exists(owner, repo, path):
        missing_files.append(path)
    else:
        dates_review.append(check_commit_dates(owner, repo, path, max_date))
    
issue_body = summary_preview(missing_files, expected_files, dates_review)
title = "💬 feedback: Retos de la sesión 01"
create_issue(owner, repo, issue_body, title)